## 1. Clone Repository & Install Dependencies

In [ ]:
# Clone the repository (skip if already exists)
import os
if not os.path.exists('CMPE259_Project'):
    !git clone https://github.com/bvegaloj/CMPE259_Project.git
else:
    print("Repository already cloned, skipping...")

%cd CMPE259_Project

# Install dependencies
!pip install -q -r requirements.txt

# Install localtunnel for public URL (alternative to ngrok)
!npm install -g localtunnel

# IMPORTANT: Populate the database (it comes empty from GitHub)
print("\n📊 Setting up database...")
import sys
sys.path.insert(0, '/content/CMPE259_Project')

# Run populate_data.py with the correct Python path
import subprocess
result = subprocess.run(
    ['python', 'data/populate_data.py'],
    cwd='/content/CMPE259_Project',
    env={**os.environ, 'PYTHONPATH': '/content/CMPE259_Project'},
    capture_output=True,
    text=True
)
print(result.stdout)
if result.returncode != 0:
    print("❌ Error:", result.stderr)
else:
    print("✅ Database populated successfully!")

## 2. Set API Keys

Enter your API keys below. You can get free API keys from:
- **Groq**: https://console.groq.com/keys (14,400 requests/day free)
- **Tavily**: https://app.tavily.com (1,000 searches/month free)

In [ ]:
import os
from getpass import getpass

# Set API keys securely (won't be visible in output)
os.environ['GROQ_API_KEY'] = getpass('Enter your Groq API Key: ')
os.environ['TAVILY_API_KEY'] = getpass('Enter your Tavily API Key: ')

print("✅ API keys set successfully!")

## 3. Test Agent Directly (Optional)

Test the agent with a few queries before launching the UI.

In [ ]:
from src.database.db_manager import DatabaseManager
from src.agent.agent_orchestrator import AgentOrchestrator
from src.llm.groq_llama_model import GroqLlamaClient

# Initialize components
db = DatabaseManager(db_path='./data/sjsu_database.db')
llm = GroqLlamaClient()
agent = AgentOrchestrator(llm=llm, db_manager=db)

# Test queries
test_queries = [
    "What are the prerequisites for CMPE 259?",
    "What programs does SJSU offer in computer science?",
    "When is the application deadline for Fall 2025?"
]

for query in test_queries:
    print(f"\n{'='*80}")
    print(f"Q: {query}")
    print('='*80)
    response = agent.run(query)
    print(response)

print("\n✅ Agent is working correctly!")

## 4. Launch Streamlit UI with Public URL (Option 1: localtunnel)

⚠️ **IMPORTANT**: Choose EITHER Option 1 (localtunnel) OR Option 2 (ngrok) below - DO NOT run both!

**Option 1 uses localtunnel** (free, requires browser verification). If you choose this option, run cells 7 and 8 below.

In [ ]:
%%bash --bg
# Start Streamlit in the background
streamlit run launch_ui.py --server.port 8501 --server.headless true > /dev/null 2>&1

In [ ]:
import time

# Wait for Streamlit to start
print("⏳ Starting Streamlit...")
time.sleep(10)

# Create public URL with localtunnel
print("🌐 Creating public URL...\n")
!npx localtunnel --port 8501 &

time.sleep(5)

print("\n" + "="*80)
print("✅ Streamlit UI is running!")
print("="*80)
print("\n📱 IMPORTANT: Copy the URL shown above (e.g., https://xxxxx.loca.lt)")
print("\n⚠️  FIRST-TIME SETUP:")
print("   1. Open the localtunnel URL in your browser")
print("   2. You'll see a warning page asking to verify the tunnel")
print("   3. Click 'Click to Continue' or enter the IP shown")
print("   4. After verification, you'll see the Streamlit app!")
print("\n💡 The verification page is NORMAL - it's localtunnel's security feature.")
print("   After the first visit, you can use the URL directly.")
print("\n⏰ Keep this cell running. Stopping it will close the tunnel.")
print("="*80)

## 4. Launch Streamlit UI with Public URL (Option 2: ngrok - RECOMMENDED)

⚠️ **IMPORTANT**: Choose EITHER Option 1 (localtunnel above) OR Option 2 (ngrok below) - DO NOT run both!

**Option 2 uses ngrok** (more stable, requires free account at https://ngrok.com).

This option is RECOMMENDED because it:
- Automatically kills any old Streamlit processes
- Validates API keys before starting
- Waits for Streamlit to be fully ready
- Shows errors immediately if something fails

In [ ]:
# Install pyngrok
!pip install -q pyngrok
from pyngrok import ngrok
import time
import requests
from getpass import getpass
import os

# Check API keys first
print("🔑 Checking API keys...")
if not os.getenv('GROQ_API_KEY'):
    print("❌ ERROR: GROQ_API_KEY not set! Run Cell 2 first to set your API keys.")
    raise ValueError("API keys not configured")
if not os.getenv('TAVILY_API_KEY'):
    print("❌ ERROR: TAVILY_API_KEY not set! Run Cell 2 first to set your API keys.")
    raise ValueError("API keys not configured")
print("✅ API keys are set")

# Set your ngrok authtoken
ngrok_token = getpass('Enter your ngrok authtoken: ')
!ngrok config add-authtoken {ngrok_token}

# Kill any existing processes
!pkill -f streamlit
!pkill -f ngrok
time.sleep(3)

# Start Streamlit with proper configuration
print("🚀 Starting Streamlit...")
!streamlit run launch_ui.py \
    --server.port 8501 \
    --server.headless true \
    --server.enableCORS false \
    --server.enableXsrfProtection false \
    --server.address 0.0.0.0 \
    > streamlit.log 2>&1 &

# Wait for Streamlit to be ready
print("⏳ Waiting for Streamlit to start...")
for i in range(30):
    try:
        response = requests.get("http://localhost:8501/_stcore/health")
        if response.status_code == 200:
            print(f"✅ Streamlit is ready! (took {i+1} seconds)")
            break
    except:
        pass
    time.sleep(1)
else:
    print("⚠️  Warning: Streamlit might not be fully ready.")
    print("📋 Checking logs for errors...")
    !tail -20 streamlit.log

# Create public URL
print("\n🌐 Creating public tunnel...")
public_url = ngrok.connect(8501)
print(f"\n{'='*80}")
print(f"✅ Success! Your Streamlit app is live!")
print(f"{'='*80}")
print(f"\n🌐 Access your app at: {public_url}")
print(f"\n💡 Keep this cell running. Stopping it will close the tunnel.")
print(f"\n📋 If you see a blank page:")
print(f"   1. Wait 10 more seconds and refresh")
print(f"   2. Check the debug cell below for errors")
print(f"   3. Make sure your API keys are set correctly")
print(f"{'='*80}")

## 5. Check Logs (if needed)

In [ ]:
# View Streamlit logs if there are any issues
!tail -50 streamlit.log

## 6. Stop Streamlit (when done)

In [ ]:
# Stop Streamlit and close tunnel
!pkill -f streamlit
!pkill -f localtunnel
print("✅ Streamlit stopped")

In [ ]:
# Debug: Check if everything is set up correctly
print("🔍 System Check:")
print("="*80)

# Check if Streamlit is running
print("\n1. Streamlit Process:")
!ps aux | grep streamlit | grep -v grep

# Check if database exists
print("\n2. Database File:")
!ls -lh ./data/sjsu_database.db 2>/dev/null || echo "❌ Database file not found!"

# Check API keys
import os
print("\n3. API Keys Status:")
print(f"   GROQ_API_KEY: {'✅ Set' if os.getenv('GROQ_API_KEY') else '❌ Not Set'}")
print(f"   TAVILY_API_KEY: {'✅ Set' if os.getenv('TAVILY_API_KEY') else '❌ Not Set'}")

# Check if port is accessible
print("\n4. Port 8501 Status:")
!netstat -tuln | grep 8501 || echo "Port 8501 not listening"

# Check streamlit logs for errors
print("\n5. Recent Streamlit Logs:")
print("="*80)
!tail -30 streamlit.log

print("\n" + "="*80)
print("💡 Troubleshooting:")
print("   - If database is missing: The sjsu_database.db should be in the repo")
print("   - If API keys not set: Re-run cell 2")
print("   - If blank page persists: Try opening the ngrok URL in an incognito window")
print("="*80)

In [ ]:
# Quick database verification
import sqlite3

db_path = './data/sjsu_database.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

print("📊 Database Content Verification:")
print("="*80)

tables = ['programs', 'prerequisites', 'faqs', 'deadlines', 'campus_resources', 
          'student_clubs', 'scholarships', 'admission_requirements']

for table in tables:
    cursor.execute(f"SELECT COUNT(*) FROM {table}")
    count = cursor.fetchone()[0]
    print(f"{table:25} {count:3} records")

print("\n" + "="*80)
print("Sample FAQ:")
cursor.execute("SELECT question, answer FROM faqs LIMIT 1")
row = cursor.fetchone()
if row:
    print(f"Q: {row[0]}")
    print(f"A: {row[1][:100]}...")
else:
    print("❌ No FAQs found!")

conn.close()

## ⚠️ IMPORTANT: If you see a blank page after database population

The Streamlit app caches data. After populating the database:
1. **Stop any running Streamlit** (run the "Stop Streamlit" cell above)
2. **Re-run the ngrok cell** (Cell 10) to restart Streamlit with fresh data
3. The new ngrok URL should now show the working app with data